## Gait Video Study 
### Creating the summary statistics file for the traditional ML algorithms on task/subject generalization frameworks 
We use the summary statistics as CoV, range and asymmetry between the right and left limbs as the features to input to the traditional models requiring fixed size 1D input for each training/testing set sample.

In [5]:
import numpy as np
import cv2
import os
import glob
import pandas as pd
import time
import shutil
import scipy
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, HTML

In [6]:
path = 'C:\\Users\\Rachneet Kaur\\Box\\Gait Video Project\\GaitVideoData\\video\\'
data_path = path + 'downsampled_strides\\'
labels_path = path + 'labels.csv'

labels = pd.read_csv(labels_path, index_col= 0)
display(labels.head())

,cohort,trial,scenario,video,PID,stride_number,key,frame_count,label
0,HOA,BW,SLWT,GVS_212_T_T1,212,1,GVS_212_T_T1_1,46,0
1,HOA,BW,SLWT,GVS_212_T_T1,212,2,GVS_212_T_T1_2,39,0
2,HOA,BW,SLWT,GVS_212_T_T1,212,3,GVS_212_T_T1_3,56,0
3,HOA,BW,SLWT,GVS_212_T_T1,212,4,GVS_212_T_T1_4,53,0
4,HOA,BW,SLWT,GVS_212_T_T1,212,5,GVS_212_T_T1_5,44,0


In [11]:
#Making the summary statistics dataframe 
#It will have all columns same as labels.csv and 90 extra columns for 36 CoV, 36 range and 18 asymmetry features
#We can then use summary statistics dataframe for all frameworks with the traditional algorithms 

keys = labels['key'] #Using the key representing the video and stride as the unique identifier
markers = ['hip', 'knee', 'ankle', 'heel', 'toe 1', 'toe 2']
order = ['right hip', 'right knee', 'right ankle', 'left hip', 'left knee', 'left ankle', 'left toe 1', 'left toe 2', \
         'left heel', 'right toe 1', 'right toe 2', 'right heel']
coordinates = [o + '-'+ y for o in order for y in ['x', 'y', 'z']]
coordinates_asymmetry = [m + '-' + y for m in markers for y in ['x', 'y', 'z']] #Defining the column names for the asymmetry
right_side_markers = ['right ' + c_a for c_a in coordinates_asymmetry] #Defining the right side markers 
left_side_markers = ['left ' + c_a for c_a in coordinates_asymmetry] #Defining the left side markers 
#Columns for the summary features dataframe 
feature_cols = [c + '-'+ y for y in ['CoV', 'range']  for c in coordinates] + [c_a + '-asymmetry' for c_a in coordinates_asymmetry]
#Summary feature dataframe with index as keys and columns as 90 summary statistics (36 CoV, 36 range and 18 asymmetry)
features_dataframe = pd.DataFrame(columns = feature_cols, index = keys)

In [12]:
#Using the key as the unique identifier to loop through each stride in the dataset
for key in keys: 
    print (key)
    stride = pd.read_csv(data_path+key+'.csv', index_col = 0) 
#     display(stride.head())
    stride_cov = stride.std()/stride.mean() #Computing the stride's coefficient of variation across all 36 features 
    stride_range = stride.max()-stride.min() #Stride's range across all 36 features 
    #Stride's asymmetry across left and right side for 18 features 
    asymmetry_values = [np.abs(stride_range[x] - stride_range[y])/(0.5*(stride_range[x] + stride_range[y])) for x, y in zip(right_side_markers, left_side_markers)]
    #All the 90 summary statistics as a list 
    summary_stats = list(stride_cov.values)+list(stride_range.values)+asymmetry_values
#     print (len(summary_stats), summary_stats)
    #Assigning the row of the features dataframe with 90 values for summary statistics 
    features_dataframe.loc[key] = summary_stats

GVS_212_T_T1_1
GVS_212_T_T1_2
GVS_212_T_T1_3
GVS_212_T_T1_4
GVS_212_T_T1_5
GVS_212_T_T1_6
GVS_212_T_T1_7
GVS_212_T_T1_8
GVS_212_T_T1_9
GVS_212_T_T1_10
GVS_212_T_T1_11
GVS_212_T_T1_12
GVS_212_T_T1_13
GVS_212_T_T1_14
GVS_212_T_T1_15
GVS_212_T_T1_16
GVS_212_T_T1_17
GVS_212_T_T1_18
GVS_212_T_T1_19
GVS_212_T_T1_20
GVS_212_T_T1_21
GVS_212_T_T1_22
GVS_212_T_T1_23
GVS_212_T_T1_24
GVS_212_T_T1_25
GVS_212_T_T1_26
GVS_212_T_T1_27
GVS_212_T_T1_28
GVS_212_T_T1_29
GVS_212_T_T1_30
GVS_212_T_T1_31
GVS_212_T_T1_32
GVS_212_T_T1_33
GVS_212_T_T1_34
GVS_212_T_T1_35
GVS_212_T_T1_36
GVS_212_T_T1_37
GVS_212_T_T1_38
GVS_212_T_T1_39
GVS_212_T_T1_40
GVS_212_T_T1_41
GVS_212_T_T1_42
GVS_212_T_T1_43
GVS_212_T_T2_1
GVS_212_T_T2_2
GVS_212_T_T2_3
GVS_212_T_T2_4
GVS_212_T_T2_5
GVS_212_T_T2_6
GVS_212_T_T2_7
GVS_212_T_T2_8
GVS_212_T_T2_9
GVS_212_T_T2_10
GVS_212_T_T2_11
GVS_212_T_T2_12
GVS_212_T_T2_13
GVS_212_T_T2_14
GVS_212_T_T2_15
GVS_212_T_T2_16
GVS_212_T_T2_17
GVS_212_T_T2_18
GVS_212_T_T2_19
GVS_212_T_T2_20
GVS_212_T_

In [13]:
#After the features dataframe is ready, concatenating the features with the other information in the labels.csv 
#This data can now be use for all frameworks in traditional algorithms 
#This will have as many rows as strides in our dataset and 99 columns 
#(90 for features and 9 for information relative to the stride)
traditional_methods_dataframe = pd.concat((labels.set_index('key'), features_dataframe), axis = 1).reset_index()
#Saving to the .csv file 
traditional_methods_dataframe.to_csv(path+'traditional_methods_dataframe.csv')

In [14]:
traditional_methods_dataframe

,key,cohort,trial,scenario,video,PID,stride_number,frame_count,label,right hip-x-CoV,...,ankle-z-asymmetry,heel-x-asymmetry,heel-y-asymmetry,heel-z-asymmetry,toe 1-x-asymmetry,toe 1-y-asymmetry,toe 1-z-asymmetry,toe 2-x-asymmetry,toe 2-y-asymmetry,toe 2-z-asymmetry
0,GVS_212_T_T1_1,HOA,BW,SLWT,GVS_212_T_T1,212,1,46,0,0.0460773,...,0.604591,0.233432,0.168252,0.0362465,0.0310732,0.631236,0.495529,0.26328,0.686741,0.458813
1,GVS_212_T_T1_2,HOA,BW,SLWT,GVS_212_T_T1,212,2,39,0,0.0215279,...,0.0925554,0.472563,0.293185,0.266283,0.117045,0.7248,0.15791,0.537486,0.338966,0.228945
2,GVS_212_T_T1_3,HOA,BW,SLWT,GVS_212_T_T1,212,3,56,0,0.0343938,...,0.0589391,0.451513,0.132201,0.132919,0.105341,0.338124,0.179538,0.422522,0.188858,0.0823754
3,GVS_212_T_T1_4,HOA,BW,SLWT,GVS_212_T_T1,212,4,53,0,0.0285112,...,0.115101,0.299212,0.00191031,0.0382431,0.0274051,0.150478,0.143856,0.0392329,0.32659,0.165196
4,GVS_212_T_T1_5,HOA,BW,SLWT,GVS_212_T_T1,212,5,44,0,0.0252132,...,0.311598,0.0793935,0.000534968,0.307031,0.0887779,0.117577,0.291998,0.254334,0.00535783,0.379653
5,GVS_212_T_T1_6,HOA,BW,SLWT,GVS_212_T_T1,212,6,40,0,0.0361427,...,0.0654001,0.507203,0.0667156,0.2723,0.526079,0.257474,0.0830628,0.431713,0.200159,0.0266662
6,GVS_212_T_T1_7,HOA,BW,SLWT,GVS_212_T_T1,212,7,40,0,0.0376991,...,0.0781821,0.158197,0.198645,0.0203465,0.107169,0.106186,0.0698676,0.104036,0.0360423,0.13748
7,GVS_212_T_T1_8,HOA,BW,SLWT,GVS_212_T_T1,212,8,47,0,0.0387768,...,0.309192,0.296529,0.272024,0.421301,0.237767,0.0373363,0.460409,0.138532,0.0243646,0.371069
8,GVS_212_T_T1_9,HOA,BW,SLWT,GVS_212_T_T1,212,9,56,0,0.0475081,...,0.0564788,0.256663,0.116658,0.0987656,0.295696,0.456175,0.253875,0.328332,0.297199,0.177619
9,GVS_212_T_T1_10,HOA,BW,SLWT,GVS_212_T_T1,212,10,46,0,0.0369667,...,0.0173921,0.523241,0.549748,0.0216528,0.688522,0.228129,0.277317,0.169427,0.196002,0.2523
